<a href="https://colab.research.google.com/github/hahahonline/YOLOv4-Deteccao-de-objetos-Videos-Darknet/blob/main/(meu)_YOLO_v4_Detec%C3%A7%C3%A3o_de_objetos_V%C3%ADdeos_Darknet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detectando objetos em videos com YOLOv4

## Etapa 1 - Download do Darknet

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
%cd darknet

In [ ]:
ls

## Etapa 2 - Compilando a biblioteca

In [ ]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(device_name)

In [ ]:
!make

## Etapa 3 - Carregando os pesos do modelo pré-treinado

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

In [ ]:
#este código está descompactando um arquivo zip localizado no caminho especificado 
#e extraindo todo o seu conteúdo para o diretório atual.
import zipfile
path = "/content/gdrive/My Drive/Cursos/YOLO/modelo_YOLOv4.zip"
zip_object = zipfile.ZipFile(file=path, mode="r")
zip_object.extractall("./")
zip_object.close()

## Etapa 4 - Carregando o vídeo

### 4.1 - De uma url

In [ ]:
#você pode carregar o video dessa URL ou do google drive (bloco abaixo)
#!wget https://github.com/gabevr/yolo/raw/master/videos/video_teste02.mp4

### 4.2 - Do Google Drive


In [ ]:
#Esse comando usa a ferramenta cp do sistema operacional Unix para copiar um arquivo de um diretório do Google Drive para o diretório atual (./), que é o diretório raiz do ambiente de execução atual no Colab.
#O ./ é um caminho relativo que indica o diretório atual em que o comando está sendo executado.
!cp /content/gdrive/My\ Drive/Cursos/YOLO/videos/uniceub.mp4 ./

In [ ]:
ls

## Etapa 5 - Realizando a detecção em vídeo

In [ ]:
#detector demo = executar o modelo YOLO em um arquivo de vídeo ou webcam ao vivo para detectar objetos em tempo real
#yolov4.cfg = estrutura da rede neural
#yolov4.weights = pesos 
#dont_show = pra nao mostrar o processamento
#indica que o Darknet deve usar o dispositivo de entrada de vídeo número 0, que é geralmente a webcam padrão do computador. (não vai funcionar pq eu passei um video na entrada)
!./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights -dont_show uniceub.mp4 -i 0 -out_filename resultado_ceub.avi

In [ ]:
#mostrando o tamanho do arquivo
!du -h resultado.avi

## Etapa 6 - Visualizando o resultado

In [ ]:
!cp ./resultado.avi /content/gdrive/My\ Drive/Cursos/YOLO/videos/resultado1.avi

## Especificando um threshold

In [ ]:
#você pode carregar o video dessa URL ou do seu google drive (bloco abaixo)
#!wget https://github.com/gabevr/yolo/raw/master/videos/video_rua01.mp4

In [ ]:
#faz uma copia do video do meu diretorio do drive  
!cp /content/gdrive/My\ Drive/Cursos/YOLO/videos/resultado_uniceub.avi ./

In [ ]:
#detector demo = executar o modelo YOLO em um arquivo de vídeo ou webcam ao vivo para detectar objetos em tempo real
#yolov4.cfg = estrutura da rede neural
#yolov4.weights = pesos 
#dont_show = pra nao mostrar o processamento
#thresh = nivel de confiança da detecção de objeto 
!./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights -dont_show resultado_uniceub.avi -out_filename resultado_uniceub2.avi -thresh 0.2

In [ ]:
#copiando o video na pasta do google drive
!cp ./resultado_uniceub2.avi /content/gdrive/My\ Drive/Cursos/YOLO/videos/resultado_uniceub2.avi